In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
training_data_gen = ImageDataGenerator(rotation_range=0.2,shear_range=0.2,
zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,
validation_split = 0.2) #created 5 new data points or images from an existing image.
#rescaling because the scale of the original image is way too large.

In [4]:
batchsize = 8

In [5]:
training_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset = 'training')

validation_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset = 'validation')
#we need to give a particular size to the image then only our model will work.

Found 64244 images belonging to 2 classes.
Found 16060 images belonging to 2 classes.


In [6]:
test_data_gen = ImageDataGenerator(rotation_range=0.2)

test_data = test_data_gen.flow_from_directory(r'PreparingData\Test',
target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4594 images belonging to 2 classes.


In [7]:
#Making the base of the model
model_base = InceptionV3(include_top= False,weights='imagenet',
input_tensor=Input(shape=(80,80,3)))
#3 due to lightning conditions

In [8]:
my_model = model_base.output


In [9]:
model_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[

In [10]:
my_model = Flatten()(my_model)
#deep learning starts from here
my_model = Dense(64,activation='relu')(my_model)
my_model = Dropout(0.5)(my_model)
my_model = Dense(2,activation='softmax')(my_model)
#since this is a categorical model so activation will be softmax.
 #just put those 2 neurons(eye_state)
#drop out to prevent overfitting
#activation= relu means take only important data you can say.

In [11]:
#final model
model = Model(inputs=model_base.input, outputs =my_model)
#since we donot want to train the model from scratch so
for layer in model_base.layers:
    layer.trainable = False

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint(r'Model\model.h5',monitor='val_loss',save_best_only=True,verbose=3)
#save the best model only based on validation loss
earlystop = EarlyStopping(patience=7,monitor='val_loss',verbose=3,restore_best_weights=True)

rateofLearning = ReduceLROnPlateau(patience=7,verbose=3,monitor='val_loss',)

In [14]:
callbacks = [checkpoint,earlystop,rateofLearning]

In [15]:
#compiling the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(training_data,steps_per_epoch=training_data.samples//batchsize,
validation_data = validation_data,
validation_steps = validation_data.samples//batchsize,
callbacks=callbacks,
epochs = 20)

C:\Users\HP\AppData\Local\Temp/ipykernel_9768/2185305659.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(training_data,steps_per_epoch=training_data.samples//batchsize,


Epoch 1/20
8030/8030 [==============================] - ETA: 0s - loss: 0.2064 - accuracy: 0.9192
Epoch 00001: val_loss improved from inf to 0.18059, saving model to Model\model.h5
8030/8030 [==============================] - 3162s 393ms/step - loss: 0.2064 - accuracy: 0.9192 - val_loss: 0.1806 - val_accuracy: 0.9233 - lr: 0.0010
Epoch 2/20
8030/8030 [==============================] - ETA: 0s - loss: 0.1802 - accuracy: 0.9290
Epoch 00002: val_loss did not improve from 0.18059
8030/8030 [==============================] - 2622s 326ms/step - loss: 0.1802 - accuracy: 0.9290 - val_loss: 0.2080 - val_accuracy: 0.9110 - lr: 0.0010
Epoch 3/20
8030/8030 [==============================] - ETA: 0s - loss: 0.1687 - accuracy: 0.9352
Epoch 00003: val_loss improved from 0.18059 to 0.17932, saving model to Model\model.h5
8030/8030 [==============================] - 2697s 336ms/step - loss: 0.1687 - accuracy: 0.9352 - val_loss: 0.1793 - val_accuracy: 0.9266 - lr: 0.0010
Epoch 4/20
8030/8030 [==========

In [17]:
#Model Evaluation for training
training_loss, training_accuracy = model.evaluate(training_data)
print(training_loss)
print(training_accuracy)

 401/8031 [>.............................] - ETA: 29:23 - loss: 0.1553 - accuracy: 0.9454- ETA: 29:4 - ETA: 29:25 - loss: 0.1553 - accuracy: 0

KeyboardInterrupt: 

In [17]:
#Model Evaluation for validation
validation_loss, validation_accuracy = model.evaluate(validation_data)
print(validation_loss)
print(validation_accuracy)

4015/4015 [==============================] - 600s 149ms/step - loss: 0.1924 - accuracy: 0.9249
0.19236500561237335
0.9249066114425659


In [18]:
#Model Evaluation for test
test_loss, test_accuracy = model.evaluate(test_data)
print(test_loss)
print(test_accuracy)

1149/1149 [==============================] - 155s 134ms/step - loss: 102.7964 - accuracy: 0.4867
102.79637145996094
0.48672181367874146
